In [1]:
!mkdir -p /mnt/data/mydataset/train_img
!mkdir -p /mnt/data/mydataset/aligned_img
!mkdir -p /mnt/data/mydataset/test_img
!mkdir -p /mnt/data/mydataset/test_aligned_img
#data mv tock  样本人 tock
#data mv trevor 样本人脸 trevor


# Preprocess the raw images  
jupyter 对shell的多线线程的支持程度不明确，若要使用多线程需要进入容器shell执行
这里对图片做归一化处理，框选出人脸部分，调整到统一大小，做选择和对齐变换。

In [2]:
#!for N in {1..8}; do ~/openface/util/align-dlib.py ./data/mydataset/raw/sj align outerEyesAndNose ./output --size 96 & done >shell.log
!~/openface/util/align-dlib.py ./data/mydataset/train_img align outerEyesAndNose ./data/mydataset/aligned_img --size 96   

=== ./data/mydataset/train_img/tock/tock!90.jpg ===
=== ./data/mydataset/train_img/trevor/trevor8.jpg ===
=== ./data/mydataset/train_img/trevor/trevor3.jpg ===
=== ./data/mydataset/train_img/trevor/trevor9.jpg ===
=== ./data/mydataset/train_img/trevor/trevor5.jpg ===
=== ./data/mydataset/train_img/tock/tock11.jpg ===
=== ./data/mydataset/train_img/trevor/trevor10.jpg ===
=== ./data/mydataset/train_img/tock/tock12.jpg ===
=== ./data/mydataset/train_img/tock/tock7.jpg ===
=== ./data/mydataset/train_img/trevor/trevor13.jpg ===
=== ./data/mydataset/train_img/tock/tock2.jpg ===
=== ./data/mydataset/train_img/tock/tock3.jpg ===
=== ./data/mydataset/train_img/tock/tock!45.jpg ===
=== ./data/mydataset/train_img/tock/tock!67.jpg ===
=== ./data/mydataset/train_img/trevor/trevor7.jpg ===
=== ./data/mydataset/train_img/trevor/trevor4.jpg ===
=== ./data/mydataset/train_img/tock/tock13.jpg ===
=== ./data/mydataset/train_img/trevor/trevor1.jpg ===
=== ./data/mydataset/train_img/tock/tock9.jpg ===
===

# Generate Representations
提取特征点的luajit脚本，需要注意的是需要归类命名好资源文件， 这样输出格式化的带标签的tensor 提供给 之后的SVM学习

In [3]:
!~/openface/batch-represent/main.lua -outDir /mnt -data /mnt/data/mydataset/aligned_img

{
  data : "/mnt/data/mydataset/aligned_img"
  imgDim : 96
  model : "/root/openface/models/openface/nn4.small2.v1.t7"
  device : 1
  outDir : "/mnt"
  cache : false
  cuda : false
  batchSize : 50
}
/mnt/data/mydataset/aligned_img	
cache lotation: 	/mnt/data/mydataset/aligned_img/cache.t7	
Loading metadata from cache.	
If your dataset has changed, delete the cache file.	
nImgs: 	32	
Represent: 32/32	


# Create the Classification Model
SVM的训练速度很快，这表示他可能被动态的应用到移动设备的场景。比如零时添加一个人脸等场景。

In [4]:
! ~/openface/demos/classifier.py train .

Loading embeddings.
Training for 2 classes.
Saving classifier to './classifier.pkl'


# Classifying New Images
* 使用新的模型，这里 使用了2个人数据集（tock,trevor） 没人不同角度表情光线条件下，共17张照片， 取出其中三张作为测试集并在测试集中加入真实拍摄的陌生人。此demo中需删除无法识别人脸的图片
* 结果,很大程度上对未预处理照片找不到人脸位置，未预处理照片识别时间比较慢，置信度不高，对不同人可能出现0.8的高分。
* 总的来说这例适合工作在 ，简单背景如验证机，一对一使用如快速人工核验红黑名单人员。

In [5]:
!~/openface/demos/classifier.py infer ./classifier.pkl ./data/mydataset/test_img/*
#{tock,trevor,other}*


=== ./data/mydataset/test_img/A.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict trevor with 0.51 confidence.

=== ./data/mydataset/test_img/C1.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict trevor with 0.81 confidence.

=== ./data/mydataset/test_img/H1.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array i

# 结论 
* 一号人物测试数据为正脸，三张识别率均为0.99
* 二号人物测试数据为正脸 ，识别律分别为0.72，0.83，0.91
* 对齐测试数据的人脸在提高识别率上会起反作用。 对齐人脸除了在提高训练速度上的作用其外其他特性还有待研究


![hello world](d:/01.png)

In [10]:
!~/openface/util/align-dlib.py ./data/mydataset/test_img align outerEyesAndNose ./data/mydataset/test_aligned_img --size 96   
! mv ./data/mydataset/test_aligned_img/* ../ 

=== ./data/mydataset/test_img/tock17.jpg ===
=== ./data/mydataset/test_img/H1.jpg ===
=== ./data/mydataset/test_img/A.jpg ===
=== ./data/mydataset/test_img/tock16.jpg ===
=== ./data/mydataset/test_img/trevor18.jpg ===
=== ./data/mydataset/test_img/trevor16.jpg ===
=== ./data/mydataset/test_img/C1.jpg ===
^C
Traceback (most recent call last):
  File "/root/openface/util/align-dlib.py", line 174, in <module>
    alignMain(args)
  File "/root/openface/util/align-dlib.py", line 118, in alignMain
    skipMulti=args.skipMulti)
  File "/usr/local/lib/python2.7/dist-packages/openface/align_dlib.py", line 175, in align
    bb = self.getLargestFaceBoundingBox(rgbImg, skipMulti)
  File "/usr/local/lib/python2.7/dist-packages/openface/align_dlib.py", line 123, in getLargestFaceBoundingBox
    faces = self.getAllFaceBoundingBoxes(rgbImg)
  File "/usr/local/lib/python2.7/dist-packages/openface/align_dlib.py", line 104, in getAllFaceBoundingBoxes
    return self.detector(rgbImg, 1)
KeyboardInterrupt


In [14]:
!~/openface/demos/classifier.py infer ./classifier.pkl ./data/mydataset/test_aligned_img/*


=== ./data/mydataset/test_aligned_img/H3.png ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict tock with 0.59 confidence.

=== ./data/mydataset/test_aligned_img/Q3.png ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict tock with 0.73 confidence.

=== ./data/mydataset/test_aligned_img/tock15.png ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0`

# 测试数据 

## 2.1.1 demo测试数据
{} 在jupyershell中支持有问题

In [24]:
!~/openface/demos/classifier.py infer ~/openface/models/openface/celeb-classifier.nn4.small2.v1.pkl ~/openface/images/examples/*

/bin/sh: 1: ~/openface/demos/classifier.py infer ~/openface/models/openface/celeb-classifier.nn4.small2.v1.pkl ~/openface/images/examples/{carell,adams,lennon}*: not found


# 2.1.2 本地数据集测试 

准备好本地照片数据，X:\share\image\train_id_img_label  所有数据根据身份证号存放到文件夹中  共2967张 去掉重复2937张
* 训练并测试 训练所需时间， 
* 在测试集中，识别出指定人"王飞"的时间和精度


In [60]:
#提取特征点 
# 对齐的人脸 变少了很多 解决方法，
#不使用归一化数据
#!~/openface/util/align-dlib.py ./data/wangli/train_id_img_label align outerEyesAndNose ./data/wangli/aligned_img --size 96
import os
if os.path.exists( r"/mnt/data/wangli/train_id_img_label/cache.t7"):os.remove(r"/mnt/data/wangli/train_id_img_label/cache.t7")
!~/openface/batch-represent/main.lua -outDir /mnt/output/wangli  -data /mnt/data/wangli/train_id_img_label

{
  data : "/mnt/data/wangli/train_id_img_label"
  imgDim : 96
  model : "/root/openface/models/openface/nn4.small2.v1.t7"
  device : 1
  outDir : "/mnt/output/wangli"
  cache : false
  cuda : false
  batchSize : 50
}
/mnt/data/wangli/train_id_img_label	
cache lotation: 	/mnt/data/wangli/train_id_img_label/cache.t7	
Creating metadata for cache.	
{
  sampleSize : 
    {
      1 : 3
      2 : 96
      3 : 96
    }
  split : 0
  verbose : true
  paths : 
    {
      1 : "/mnt/data/wangli/train_id_img_label"
    }
  samplingMode : "balanced"
  loadSize : 
    {
      1 : 3
      2 : 96
      3 : 96
    }
}
running "find" on each class directory, and concatenate all those filenames into a single file containing all image paths for a given class	
now combine all the files to a single large file	
load the large concatenated list of sample paths to self.imagePath	
2937 samples found.	....................]                              ETA: 0ms | Step: 0m 0/2937 
Updating classList and imageClas

In [61]:
%time ! ~/openface/demos/classifier.py train  /mnt/output/wangli

Loading embeddings.
Training for 2937 classes.
Saving classifier to '/mnt/output/wangli/classifier.pkl'
CPU times: user 1.85 s, sys: 270 ms, total: 2.12 s
Wall time: 1min 59s


In [62]:
#无法识别出
%time !~/openface/demos/classifier.py infer /mnt/output/wangli/classifier.pkl /mnt/data/wangli/test_img/*


=== /mnt/data/wangli/test_img/Q1.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict 511111197602130425 with 0.00 confidence.

=== /mnt/data/wangli/test_img/Q2.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict 511111197602130425 with 0.00 confidence.

=== /mnt/data/wangli/test_img/Q3.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0`

# 使用归一化数据



In [67]:
#!~/openface/util/align-dlib.py ./data/wangli/train_id_img_label align outerEyesAndNose ./data/wangli/aligned_img --size 96

import os
if os.path.exists( r"/mnt/data/wangli/train_id_img_label/cache.t7"):os.remove(r"/mnt/data/wangli/train_id_img_label/cache.t7");print("del cache")
!~/openface/batch-represent/main.lua -outDir /mnt/output/wangli  -data /mnt/data/wangli/aligned_img

{
  data : "/mnt/data/wangli/aligned_img"
  imgDim : 96
  model : "/root/openface/models/openface/nn4.small2.v1.t7"
  device : 1
  outDir : "/mnt/output/wangli"
  cache : false
  cuda : false
  batchSize : 50
}
/mnt/data/wangli/aligned_img	
cache lotation: 	/mnt/data/wangli/aligned_img/cache.t7	
Creating metadata for cache.	
{
  sampleSize : 
    {
      1 : 3
      2 : 96
      3 : 96
    }
  split : 0
  verbose : true
  paths : 
    {
      1 : "/mnt/data/wangli/aligned_img"
    }
  samplingMode : "balanced"
  loadSize : 
    {
      1 : 3
      2 : 96
      3 : 96
    }
}
running "find" on each class directory, and concatenate all those filenames into a single file containing all image paths for a given class	
now combine all the files to a single large file	
load the large concatenated list of sample paths to self.imagePath	
5 samples found.	.......................]                              ETA: 0ms | Step: 0m 0/5 
Updating classList and imageClass appropriately	
 [============

In [68]:
%time ! ~/openface/demos/classifier.py train  /mnt/output/wangli


Loading embeddings.
Training for 5 classes.
Saving classifier to '/mnt/output/wangli/classifier.pkl'
CPU times: user 60 ms, sys: 20 ms, total: 80 ms
Wall time: 3.43 s

=== /mnt/data/wangli/test_img/Q1.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict 652923199908053346 with 0.22 confidence.

=== /mnt/data/wangli/test_img/Q2.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict 652923199908053346 with 0.23 confidence.

=== /mnt/data/wangli/test_img/Q3.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocess

In [69]:

%time !~/openface/demos/classifier.py infer /mnt/output/wangli/classifier.pkl /mnt/data/wangli/test_img/*


=== /mnt/data/wangli/test_img/A2.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict 652923199908053346 with 0.22 confidence.

=== /mnt/data/wangli/test_img/C3.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict 652923199908053346 with 0.25 confidence.

=== /mnt/data/wangli/test_img/F4.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0`

# 结论  

在2次训练后
1 使用2973张网力图片 其中包含目标图片，不经过归一化处理

2 使用5张归一化后的图片，包含一张目标图片

以上均不能识别成功
1：N 的分类识别方法需要数据支持。网力的身份证照片是81*100 96dpi 1张 不足以用openface 识别



# 用网络数据集 测试 openface 的识别速度
* 102人的已知照片  每个人有1-3张作为训练数据，测试算法对其中一人的 识别速度
* 使用openface.ipynb notebook 打标签测试数据集

In [2]:
# 数据集  位置 D:\docker\mnt\openface\data\mydataset\train_img
!~/openface/util/align-dlib.py ./data/mydataset/train_img align outerEyesAndNose ./data/mydataset/aligned_img --size 96 

=== ./data/mydataset/train_img/adrian/adrian3.jpg ===
=== ./data/mydataset/train_img/merilyn/merilyn1.jpg ===
=== ./data/mydataset/train_img/scott/scott3.jpg ===
=== ./data/mydataset/train_img/pkyle/pkyle2.jpg ===
=== ./data/mydataset/train_img/olive/olive3.jpg ===
=== ./data/mydataset/train_img/dsmith/dsmith3.jpg ===
=== ./data/mydataset/train_img/miranda/miranda4.jpg ===
=== ./data/mydataset/train_img/gfindley/gfindley1.jpg ===
=== ./data/mydataset/train_img/chris/chris3.jpg ===
=== ./data/mydataset/train_img/bfegan/bfegan2.jpg ===
=== ./data/mydataset/train_img/michael/michael6.jpg ===
=== ./data/mydataset/train_img/david_imray/david_imray1.jpg ===
=== ./data/mydataset/train_img/simon/simon5.jpg ===
=== ./data/mydataset/train_img/pat/pat3.jpg ===
=== ./data/mydataset/train_img/paul/paul14.jpg ===
=== ./data/mydataset/train_img/heather/heather4.jpg ===
=== ./data/mydataset/train_img/lynn/lynn2.jpg ===
=== ./data/mydataset/train_img/adrian/adrian2.jpg ===
=== ./data/mydataset/train_im

=== ./data/mydataset/train_img/peter/peter12.jpg ===
=== ./data/mydataset/train_img/jenni/jenni2.jpg ===
=== ./data/mydataset/train_img/mark/mark4.jpg ===
=== ./data/mydataset/train_img/hin/hin2.jpg ===
=== ./data/mydataset/train_img/barry/barry1.jpg ===
=== ./data/mydataset/train_img/kieran/kieran3.jpg ===
=== ./data/mydataset/train_img/george/george1.jpg ===
=== ./data/mydataset/train_img/derek/derek2.jpg ===
=== ./data/mydataset/train_img/trevor/trevor18.jpg ===
=== ./data/mydataset/train_img/marie/marie13.jpg ===
=== ./data/mydataset/train_img/joanna/joanna1.jpg ===
=== ./data/mydataset/train_img/lynn/lynn3.jpg ===
=== ./data/mydataset/train_img/dbell/dbell1.jpg ===
=== ./data/mydataset/train_img/alister/alister1.jpg ===
=== ./data/mydataset/train_img/nick/nick1.jpg ===
=== ./data/mydataset/train_img/heatherg/heatherg.jpg ===
=== ./data/mydataset/train_img/dbell/dbell3.jpg ===
=== ./data/mydataset/train_img/john_thom/john_thom1.jpg ===
=== ./data/mydataset/train_img/jsheenang/jshee

In [11]:
!rm /mnt/data/mydataset/aligned_img/cache.t7 
!~/openface/batch-represent/main.lua -outDir /mnt -data /mnt/data/mydataset/aligned_img

{
  data : "/mnt/data/mydataset/aligned_img"
  imgDim : 96
  model : "/root/openface/models/openface/nn4.small2.v1.t7"
  device : 1
  outDir : "/mnt"
  cache : false
  cuda : false
  batchSize : 50
}
/mnt/data/mydataset/aligned_img	
cache lotation: 	/mnt/data/mydataset/aligned_img/cache.t7	
Creating metadata for cache.	
{
  sampleSize : 
    {
      1 : 3
      2 : 96
      3 : 96
    }
  split : 0
  verbose : true
  paths : 
    {
      1 : "/mnt/data/mydataset/aligned_img"
    }
  samplingMode : "balanced"
  loadSize : 
    {
      1 : 3
      2 : 96
      3 : 96
    }
}
running "find" on each class directory, and concatenate all those filenames into a single file containing all image paths for a given class	
now combine all the files to a single large file	
load the large concatenated list of sample paths to self.imagePath	
221 samples found.	.....................]                              ETA: 0ms | Step: 0m 0/221 
Updating classList and imageClass appropriately	
 [============

In [12]:
#99 人的分类总数
%time ! ~/openface/demos/classifier.py train .


Loading embeddings.
Training for 99 classes.
Saving classifier to './classifier.pkl'
CPU times: user 30 ms, sys: 30 ms, total: 60 ms
Wall time: 2.82 s


#  统计结果
无法检测 超过67度的侧脸，跳过
* 2 个实验对象 均无法 得出结果 是否是每个人的样本数量不够导致？

In [13]:
%time !~/openface/demos/classifier.py infer ./classifier.pkl ./data/mydataset/test_img/tock*
%time !~/openface/demos/classifier.py infer ./classifier.pkl ./data/mydataset/test_img/tracy*


=== ./data/mydataset/test_img/tock!22.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict tracy with 0.03 confidence.

=== ./data/mydataset/test_img/tock!45.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict stewart with 0.04 confidence.

=== ./data/mydataset/test_img/tock!67.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check

/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict lynn with 0.03 confidence.

=== ./data/mydataset/test_img/tracy6.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict marie with 0.04 confidence.

=== ./data/mydataset/test_img/tracy7.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict mari

# 将每个人的训练数据增加到9 继续尝试 不足九个则只训练不测试
手动删除之前的文件
* 训练用时3.38s 427张图片

In [14]:
!~/openface/util/align-dlib.py ./data/mydataset/train_img align outerEyesAndNose ./data/mydataset/aligned_img --size 96 
!rm /mnt/data/mydataset/aligned_img/cache.t7 
!~/openface/batch-represent/main.lua -outDir /mnt -data /mnt/data/mydataset/aligned_img
%time ! ~/openface/demos/classifier.py train .

=== ./data/mydataset/train_img/pat/pat15.jpg ===
=== ./data/mydataset/train_img/mnicholson/mnicholson1.jpg ===
=== ./data/mydataset/train_img/iroy/iroy3.jpg ===
=== ./data/mydataset/train_img/barry/barry11.jpg ===
=== ./data/mydataset/train_img/olive/olive2.jpg ===
=== ./data/mydataset/train_img/tracy/tracy8.jpg ===
=== ./data/mydataset/train_img/pat/pat6.jpg ===
=== ./data/mydataset/train_img/tock/tock8.jpg ===
=== ./data/mydataset/train_img/pkyle/pkyle1.jpg ===
=== ./data/mydataset/train_img/joanna/joanna3.jpg ===
=== ./data/mydataset/train_img/marie/marie14.jpg ===
=== ./data/mydataset/train_img/mnicholson/mnicholson4.jpg ===
=== ./data/mydataset/train_img/barry/barry13.jpg ===
=== ./data/mydataset/train_img/adrian/adrian1.jpg ===
=== ./data/mydataset/train_img/hin/hin5.jpg ===
=== ./data/mydataset/train_img/jim/jim2.jpg ===
=== ./data/mydataset/train_img/bfegan/bfegan3.jpg ===
=== ./data/mydataset/train_img/derek/derek3.jpg ===
=== ./data/mydataset/train_img/nick/nick11.jpg ===
===

=== ./data/mydataset/train_img/dhawley/dhawley3.jpg ===
=== ./data/mydataset/train_img/meggan/meggan1.jpg ===
=== ./data/mydataset/train_img/miranda/miranda3.jpg ===
=== ./data/mydataset/train_img/david_imray/david_imray1.jpg ===
=== ./data/mydataset/train_img/andrew/andrew13.jpg ===
=== ./data/mydataset/train_img/mmanson/mmanson4.jpg ===
=== ./data/mydataset/train_img/amellanby/amellanby5.jpg ===
=== ./data/mydataset/train_img/lisa/lisa9.jpg ===
=== ./data/mydataset/train_img/alison/alison3.jpg ===
=== ./data/mydataset/train_img/pkyle/pkyle4.jpg ===
=== ./data/mydataset/train_img/peter/peter7.jpg ===
=== ./data/mydataset/train_img/heather/heather1.jpg ===
=== ./data/mydataset/train_img/simon/simon6.jpg ===
=== ./data/mydataset/train_img/stephen/stephen!67.jpg ===
=== ./data/mydataset/train_img/kirsty/kirsty18.jpg ===
=== ./data/mydataset/train_img/lynn/lynn1.jpg ===
=== ./data/mydataset/train_img/tock/tock17.jpg ===
=== ./data/mydataset/train_img/irene/irene1.jpg ===
=== ./data/mydata

=== ./data/mydataset/train_img/mmanson/mmanson3.jpg ===
=== ./data/mydataset/train_img/trevor/trevor2.jpg ===
=== ./data/mydataset/train_img/andrew/andrew12.jpg ===
=== ./data/mydataset/train_img/bfegan/bfegan2.jpg ===
=== ./data/mydataset/train_img/jenni/jenni12.jpg ===
=== ./data/mydataset/train_img/andrew/andrew9.jpg ===
=== ./data/mydataset/train_img/peter/peter1.jpg ===
=== ./data/mydataset/train_img/simon/simon7.jpg ===
=== ./data/mydataset/train_img/catherine/catherine7.jpg ===
=== ./data/mydataset/train_img/annanena/annanena1.jpg ===
=== ./data/mydataset/train_img/amellanby/amellanby15.jpg ===
=== ./data/mydataset/train_img/heather/heather2.jpg ===
=== ./data/mydataset/train_img/kay/kay8.jpg ===
=== ./data/mydataset/train_img/dpearson/dpearson8.jpg ===
=== ./data/mydataset/train_img/marieg/marieg.jpg ===
=== ./data/mydataset/train_img/adrian/adrian2.jpg ===
=== ./data/mydataset/train_img/amellanby/amellanby16.jpg ===
=== ./data/mydataset/train_img/michael/michael17.jpg ===
=== 

In [15]:
%time !~/openface/demos/classifier.py infer ./classifier.pkl ./data/mydataset/test_img/tock*
%time !~/openface/demos/classifier.py infer ./classifier.pkl ./data/mydataset/test_img/tracy*


=== ./data/mydataset/test_img/tock!22.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict tock with 0.08 confidence.

=== ./data/mydataset/test_img/tock!67.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict tock with 0.04 confidence.

=== ./data/mydataset/test_img/tock1.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that 


* 12 次 识别 tock 时间 90ms 平均 7.5ms/次  所有12张都识别到tock 置信分  最低0.04 为侧脸67度 最高0.10 光线灰度变化没有明显对得分产生影响
* 4次tracy 用时70ms  所有检测结果均正确    置信分 最高 0.09 最低0.08 

# 训练的图片
## tock
 <img src="./doc/tock20181024164019.png"  align=center />
 
## tracy
 <img src="./doc/tracy20181024164236.png" align=center />
 
 ## 测试图片 
 ## tock
  <img src="./doc/tock_test.png"  align=center />
  
 ## tracy
   <img src="./doc/tracy_test.png"  align=center />

# 对照组 一下是不能识别的个体
* 个体 noman1 为没有经过训练的 
置信分<0.03

In [18]:
%time !~/openface/demos/classifier.py infer ./classifier.pkl ./data/mydataset/test_img/noman*



=== ./data/mydataset/test_img/noman1.jpg ===
/root/.local/lib/python2.7/site-packages/sklearn/preprocessing/label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
Predict kieran with 0.03 confidence.
CPU times: user 30 ms, sys: 30 ms, total: 60 ms
Wall time: 2.71 s


# 结论

openface 的人脸分类需要提供 3-9张的基础数据作为分类训练。
预测置信分 ：Predict tracy with 0.09 confidence   分数低的原因可能是 与分类数量有关吗？ 分类的总量越大，分数越低吗？